## Parameters - copy from main 

In [ ]:
M = 1::Int; # number of simulations  

L = 60.0::Float64; # length of the periodic-boundaries stripe (in 10^-5 m)
R = 2.0::Float64;   # radius of cells (in 10^-5 m)
nCell = 7::Int;   # number of cells 

mu = 5.0e10::Float64; # friction coefficient. 
fMot = 8.3e-9*mu::Float64;  # motile force.
fCont = 5.5*fMot::Float64;  # contact force maximum.
sigma = 4.5e-8*mu::Float64; # standard deviation of Langevin gaussian noise.  

f0 = 100.0::Float64;

nTimeStep = 600::Int;  # number of time steps.
dt = 0.01;  # numerical time step size. Should stay < 0.5. Real time step size = dt*Tm with Tm = muR/fMot.
tInt = 1.0*60;

directedRot = false;  # Don't put true if brokenPattern = true!
randomPol = false;

#  Integer scientific notation of mu for folder name. 
muA = mu; muB = 0;
while muA>=100
    muA /=10;
    muB+=1;
end

folder = "D:\\FolderName"::AbstractString;  # path for creating directory
folder = string(folder, "\\t$(nTimeStep)_dt$(dt)_tInt$(tInt)\\L$(Int(round(L)))_R$(Int(round(R)))\\nCell$(nCell)\\fMot$(Int(round(fMot)))_fCont$(Int(round(fCont)))_sigma$(Int(round(sigma)))_mu$(Int(round(muA)))e$(muB)\\f0_$(Int(round(f0)))")

## Useful functions

In [ ]:
function makeFolder(dir::AbstractString)  # Create the input directory if it doesnt exist 
    if !isdir(dir)
        mkpath(dir)
    end
end 

## Code 

In [ ]:
xC = R;
LC = L/xC; 
tC = mu*R*10.0^(-5)/fMot;

#####################
for m= 1:M
   println("Exp $m out of $M")
    
                    ##### Create Folder for output graphs #####
    dir = string(folder,"\\exp$(dec(m,2))");
    ##dir = folder;
    dirPlot = string(dir, "\\plot");
    makeFolder(dirPlot);

    using PyPlot
    color = zeros(nCell,3);

    for t=1:nTimeStep

                    ##### Prepare plots path ##### 
        println("t = $t out of $nTimeStep ... \n")
        oFileName = "DataStep_$(dec(t,4)).csv";
        oFilePath = string(dir,'\\',oFileName);
        imString = string(dirPlot,"\\t_$(dec(t,4)).png");
    
                    ##### Extract values for plots #####
        oArray = readcsv(oFilePath);
        posArray = oArray[:,2];
        stateArray = oArray[:,5];
        numArray = oArray[:,1];
        fCArray = oArray[:,7];

                    ##### Define colours for cells #####
        for i=1:nCell
            color[i,:] = [199, numArray[i,1]*33,numArray[i,1]*27];
            color[i,:] .%= 200.0;
            color[i,:] += 56.0;
            color[i,:] /= 256.0;
        end

                    ##### Plot #####
        xArray = LC*cos.(2.0*pi*posArray/LC)/(2.0*pi);
        yArray = LC*sin.(2.0*pi*posArray/LC)/(2.0*pi);  
        range = 0:1:100;
    
                        ### Black ring ###
        plot(LC*cos.(2.0*pi*range/100)/(2.0*pi),LC*sin.(2.0*pi*range/100)/(2.0*pi),"k",zorder = 1)  

                        ### Cell centers with scatter ###
        for i=1:nCell
            if i in wallCell
                scatter(xArray[i,1],yArray[i,1], s=250, c=[0.2,0.2,0.2], marker = "o",linewidths = 1, edgecolors = "k",zorder = 3);
            elseif stateArray[i,1] == 0.0 
                scatter(xArray[i,1],yArray[i,1], s=250, c=[color[i,:]], marker = "o", linewidths = 1, edgecolors = "k",zorder = 3);
            elseif stateArray[i,1] == -1.0
                scatter(xArray[i,1],yArray[i,1], s=250, c=[color[i,:]], marker = (3,0,360*posArray[i,1]/LC+180), linewidths = 1, edgecolors = "k", zorder = 3);
            else 
                scatter(xArray[i,1],yArray[i,1], s=250, c=[color[i,:]], marker = (3,0,360*posArray[i,1]/LC), linewidths = 1, edgecolors = "k", zorder = 3);
            end

                        ### Blue Lines for Contact forces ###
            scatter(xArray[i,1],yArray[i,1], s=0.005*(fCArray[i,1])^2, c="b", marker = (2,2,360*posArray[i,1]/LC+90),zorder = 4);
        end
    
                    ##### Plot parameters #####
        axis("square")
        PyPlot.autoscale(enable = false)
        ylim(-3.0-LC/(2.0*pi) , 3.0+LC/(2.0*pi))
        xlim(-3.0-LC/(2.0*pi) , 3.0+LC/(2.0*pi))  
        text(0.0,-2.0-LC/(2.0*pi),"t = $(round(t*dt*tC*10/3600)/10)h",horizontalalignment="center")
        axis("off")
        
                        ##### Save Figure #####
        savefig(imString)
        #savefig(imString, format = "svg")
        close()
    end
end